#### Описание стратегии торговли 

Торговая стратегия построена на анализе японских свечей и обнаружении паттерна **"Бычье поглощение" / "Медвежье поглощение"**
 в совокупности с трендовым индикатором - скользящей средней (МА).
 
![Bullish takeover](./assets/images/bullish_takeover.png "Bullish takeover") ![Bearish takeover](./assets/images/bearish_takeover.png "Bearish takeover")

Свечные модели принято подразделять на две категории:
-   **Модели разворота**: молот и повешенный, утренняя и вечерняя звезда, поглощение, харами, завеса из облаков и другие.
-   **Модели продолжения тренда**: гэп тацуки, у основания/в основании, толчок, удержание на татами и другие.

Модель торгует по рынку:
-   внутри дня с 10-00 до 23-30 мск на 10min таймфрейме
-   с переносом позиции на 1h таймфрейме 

Сигналы для входа: **buy/sell** генерируются на двух таймфреймах: `1h, 10min`.

Сигнал **buy**:
-   цена закрытия (close) последней h1 свечи выше значения MA24
-   наличие паттерна **"Бычье поглощение"** на h1
-   цена закрытия (close) последней 10min свечи выше значения MA144
-   наличие паттерна **"Бычье поглощение"** на 10min.

Сигнал **sell**:
-   цена закрытия (close) последней h1 свечи ниже значения MA24
-   наличие паттерна **"Медвежье поглощение"** на h1
-   цена закрытия (close) последней 10min свечи ниже значения MA144
-   наличие паттерна **"Медвежье поглощение"** на 10min.

Выход из рынка:
-   внутри дня по цене закрытия 10 минутной свечи в 23-30 мск.
-   на 1h таймфрейме при получении встречного разворотного паттерна.

### Модуль загрузки истории

In [ ]:
# analysis of financial markets
from moexalgo import session, Ticker
import os
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

# получаем данные для авторизации на бирже: адрес почты и пороль при регистрации на Московской бирже
username = ""
password = ""
# авторизуемся на бирже
session.authorize(username, password)

def upload_history(start_date, end_date, stock, timeframes):
    # создаем рыночный инструмент
    financial_instrument = Ticker(stock)
    # получаем историю за период
    df = financial_instrument.candles(start=start_date, end=end_date, period=timeframes)        
    
    return df

STOCK = "RUAL"
TIMEFRAMES = ["1h", "10min"]
START_DATE = "2024-07-01"
END_DATE = "2024-07-19"

df_1h = upload_history(start_date=START_DATE, end_date=END_DATE, stock=STOCK, timeframes=TIMEFRAMES[0])
df_10min = upload_history(start_date=START_DATE, end_date=END_DATE, stock=STOCK, timeframes=TIMEFRAMES[1])

In [ ]:
display(df_1h)

In [ ]:
display(df_10min)

In [ ]:
# создаем путь к файлу истории
history_path_1h = f"./history/{STOCK}/{TIMEFRAMES[0]}/{START_DATE}_{END_DATE}.csv"
history_path_10min = f"./history/{STOCK}/{TIMEFRAMES[1]}/{START_DATE}_{END_DATE}.csv"

if not os.path.exists(f"./history/{STOCK}/{TIMEFRAMES[0]}"):
    # создаем директорию
    os.makedirs(f"./history/{STOCK}/{TIMEFRAMES[0]}")
    
if not os.path.exists(f"./history/{STOCK}/{TIMEFRAMES[1]}"):
    # создаем директорию
    os.makedirs(f"./history/{STOCK}/{TIMEFRAMES[1]}")

# сохраняем историю в формат csv
df_1h.to_csv(f"{history_path_1h}", sep=",", encoding="utf-8", index=False)    
df_10min.to_csv(f"{history_path_10min}", sep=",", encoding="utf-8", index=False)

### Пользовательские функции

In [ ]:
def get_subdirectories(directory):
    return [name for name in os.listdir(directory) if os.path.isdir(os.path.join(directory, name))]


def diary_trading(signals, df, tm, quantity):
    df_rez = df
    if df_rez.shape[0] == 0:
        for index_row, row in signals.iterrows():
            if row['durations'] != None:
                row_df = []
                # таймфрейм
                row_df.append(tm)
                # дата сделки
                if tm == "1d":
                    row_df.append(dt.datetime.strptime(row['begin'], '%Y-%m-%d').strftime('%Y-%m-%d'))
                else:
                    row_df.append(dt.datetime.strptime(row['begin'], '%Y-%m-%d %H:%M:%S').strftime('%Y-%m-%d'))
                # время сделки
                if tm == "1d":
                    row_df.append("00:00:00")
                else:
                    row_df.append(dt.datetime.strptime(row['begin'], '%Y-%m-%d %H:%M:%S').strftime('%H:%M:%S'))
                # сигнал
                row_df.append(row['durations'])
                # позиция
                row_df.append(row['positions'])
                # цена открытия сделки
                if row['positions'] == 'input' or row['positions'] == 'add' or row['positions'] == 'over':
                    row_df.append(row['close'])
                else:
                    row_df.append(0)
                # цена закрытия сделки
                if row['positions'] == 'output':
                    row_df.append(row['close'])
                    # список индексов с неопределенными ценами закрытия
                    indexes = []
                    # перебираем все строки статистики
                    for ind, row_stat in df_rez.iterrows():
                        if row_stat['ClosePrice'] == 0:
                            indexes.append(ind)
                    # перебираем индексы
                    for i in indexes:
                        df_rez.iat[i, 6] = row['close']
                elif row['positions'] == 'over':
                    row_df.append(0)
                    # список индексов с неопределенными ценами закрытия
                    indexes = []
                    # перебираем все строки статистики
                    for ind, row_stat in df_rez.iterrows():
                        if row_stat['ClosePrice'] == 0:
                            indexes.append(ind)
                    # перебираем индексы
                    for i in indexes:
                        df_rez.iat[i, 6] = row['close']
                else:
                    row_df.append(0)
                # количество акций
                if row['positions'] == 'over':
                    row_df.append(df_rez.iloc[-1]["Lots"] + quantity)
                elif row['positions'] == 'output':
                    row_df.append(df_rez.iloc[-1]["Lots"])
                else:
                    row_df.append(quantity)
                # сумма сделки
                if row['positions'] == 'output' or row['positions'] == 'over':
                    row_df.append(round(row_df[-1] * row['close'], 1))
                else:
                    row_df.append(round(row_df[-1] * row['close'], 1))
                # общее количество акций
                if row['positions'] == 'add':
                    row_df.append(df_rez.iloc[-1]["Lots"] + quantity)
                elif row['positions'] == 'output':
                    row_df.append(0)
                else:
                    row_df.append(quantity)
                # записываем строку в коней датафрейма
                df_rez.loc[len(df_rez.index)] = row_df
        # расчет прибыли/убытков
        profit_loss = []
        for ind, row in df_rez.iterrows():
            closePrice = row['ClosePrice']
            openPrice = row['OpenPrice']
            if row['OpenPrice'] != 0:
                if row['Position'] == 'over':
                    contract = row['Quantity'] / 2
                else:
                    contract = row['Quantity']
                if row['Duration'] == 'buy':
                    profit_loss.append(round(contract * (closePrice - openPrice), 2))
                elif row['Duration'] == 'sell':
                    profit_loss.append(round(contract * (openPrice - closePrice), 2))
            else:
                profit_loss.append(0)
        # записываем прибыль/убыток
        df_rez['Profit'] = profit_loss
        # обрезаем датафрейм по последнему закрытию
        index_last_output = 0
        for ind, row in df_rez.iterrows():
            if row['Position'] == 'output':
                index_last_output = ind
        df_rez = df_rez.iloc[:index_last_output + 1, :]

    return df_rez


def statistics(diary, stock, tm):
    stat_dict = {"Stocks": stock, "TimeFrames": tm, "Profit": 0, "Trades": 0,
                 "TradesProfit": 0, "TradesLoss": 0, "TotalProfit": 0, "TotalLoss": 0, "ProfitRatio": 0}
    # заполняем словарь
    stat_dict["Profit"] = diary["Profit"].sum()
    stat_dict["Trades"] = diary.loc[(diary["Profit"] > 0) | (diary["Profit"] < 0), "Profit"].count()
    stat_dict["TradesProfit"] = diary.loc[diary["Profit"] > 0, "Profit"].count()
    stat_dict["TradesLoss"] = diary.loc[diary["Profit"] < 0, "Profit"].count()
    stat_dict["TotalProfit"] = diary.loc[diary["Profit"] > 0, "Profit"].sum()
    stat_dict["TotalLoss"] = diary.loc[diary["Profit"] < 0, "Profit"].sum()
    stat_dict["ProfitRatio"] = round(stat_dict["TotalProfit"] / (stat_dict["TotalLoss"] * -1), 2)

    return stat_dict


def total_statistics(total_tbl, stock):
    total_row= {"Stocks": stock, "TimeFrames": "total", "Profit": 0, "Trades": 0,
                 "TradesProfit": 0, "TradesLoss": 0, "TotalProfit": 0, "TotalLoss": 0, "ProfitRatio": 0}
    # заполняем словарь
    total_row["Profit"] = total_tbl["Profit"].sum()
    total_row["Trades"] = total_tbl["Trades"].sum()
    total_row["TradesProfit"] = total_tbl["TradesProfit"].sum()
    total_row["TradesLoss"] = total_tbl["TradesLoss"].sum()
    total_row["TotalProfit"] = total_tbl["TotalProfit"].sum()
    total_row["TotalLoss"] = total_tbl["TotalLoss"].sum()
    total_row["ProfitRatio"] = round(total_tbl["ProfitRatio"].mean(), 2)

    return total_row

### Модуль торговой стратегии

In [ ]:
import datetime as dt


def pattern_bearish_takeover(candle1, candle2):
    # если предыдущая свеча бычья
    if candle1['open'] < candle1['close']:
        if candle2['open'] >= candle1['close'] and candle2['close'] < candle1['open']:
            return True
        else:
            return False
    else:
        False


def pattern_bullish_takeover(candle1, candle2):
    # если предыдущая свеча медвежья
    if candle1['open'] > candle1['close']:
        if candle2['open'] <= candle1['close'] and candle2['close'] > candle1['open']:
            return True
        else:
            return False
    else:
        False


def strategy_takeover(df, ma, tm, add):
    # локальные переменные для торгов
    INPUT = False
    OUTPUT = True
    ADD = False
    OVER = False
    TRAND_UP = False
    TRAND_DOWN = False
    # Определяем дату начала торгов на истории
    candle_start = df.iloc[ma]
    if tm == "1d":
        trading_start_date = (dt.datetime.strptime(candle_start['begin'], '%Y-%m-%d') + dt.timedelta(
            days=1)).strftime('%Y-%m-%d')
        # получим  индексы первых свечей с даты начала торгов
        index_candel_start_trading = df.loc[df['begin'] == f'{trading_start_date}'].index[0]
    else:
        trading_start_date = (dt.datetime.strptime(candle_start['begin'], '%Y-%m-%d %H:%M:%S') + dt.timedelta(
        days=1)).strftime('%Y-%m-%d')
        # получим  индексы первых свечей с даты начала торгов
        index_candel_start_trading = df.loc[df['begin'] == f'{trading_start_date} 10:00:00'].index[0]
    # обрезаем датафремы с даты начала торгов
    signals = df.iloc[index_candel_start_trading:, :].reset_index(drop=True)

    # список сигналов 1h на всей истории
    labels_duration = []
    # список позиций 1h на всей истории
    labels_position = []

    # поиск сигналов на истории 1h
    for index, row in signals.iterrows():
        # получаем теущую и предыдущую свечки
        current_candle = row
        previous_candle = signals.iloc[index - 1]
        # если обнаружен медвежий паттерн по тренду
        if pattern_bearish_takeover(previous_candle, current_candle) and current_candle['close'] < current_candle[f'ma{ma}']:
            labels_duration.append('sell')
            # вне рынка
            if OUTPUT:
                labels_position.append('input')
                INPUT, OUTPUT, ADD, OVER = True, False, False, False
            # в рынке
            elif (INPUT or ADD or OVER) and TRAND_DOWN:
                labels_position.append('add')
                INPUT, OUTPUT, ADD, OVER = False, False, True, False
            elif (INPUT or ADD or OVER) and TRAND_UP:
                labels_position.append('over')
                INPUT, OUTPUT, ADD, OVER = False, False, False, True
            # меняем тренд
            TRAND_UP = False
            TRAND_DOWN = True

        # если обнаружен медвежий паттерн против тренда
        elif pattern_bearish_takeover(previous_candle, current_candle) and current_candle['close'] > current_candle[f'ma{ma}']:
            # вне рынка
            if OUTPUT:
                labels_duration.append(None)
                labels_position.append('output')
                INPUT, OUTPUT, ADD, OVER = False, True, False, False
            # в рынке
            elif INPUT or ADD or OVER:
                labels_duration.append('sell')
                labels_position.append('output')
                INPUT, OUTPUT, ADD, OVER = False, True, False, False

        # если обнаружен бычий паттерн по тренду
        elif pattern_bullish_takeover(previous_candle, current_candle) and current_candle['close'] > current_candle[f'ma{ma}']:
            labels_duration.append('buy')
            # вне рынка
            if OUTPUT:
                labels_position.append('input')
                INPUT, OUTPUT, ADD, OVER = True, False, False, False
            # в рынке
            elif (INPUT or ADD or OVER) and TRAND_UP:
                labels_position.append('add')
                INPUT, OUTPUT, ADD, OVER = False, False, True, False
            elif (INPUT or ADD or OVER) and TRAND_DOWN:
                labels_position.append('over')
                INPUT, OUTPUT, ADD, OVER = False, False, False, True
            # меняем тренд
            TRAND_UP = True
            TRAND_DOWN = False

        # если обнаружен бычий паттерн против тренда
        elif pattern_bullish_takeover(previous_candle, current_candle) and current_candle['close'] < current_candle[f'ma{ma}']:
            # вне рынка
            if OUTPUT:
                labels_duration.append(None)
                labels_position.append('output')
                INPUT, OUTPUT, ADD, OVER = False, True, False, False
            # в рынке
            elif INPUT or ADD or OVER:
                labels_duration.append('buy')
                labels_position.append('output')
                INPUT, OUTPUT, ADD, OVER = False, True, False, False

        # если нет паттерна
        else:
            labels_duration.append(None)
            labels_position.append('output')

    # формируем признак сгенерированных сигналов на истории 1h
    signals['durations'] = labels_duration
    # формируем признак сгенерированных позиций на истории 1h
    signals['positions'] = labels_position

    return signals

### Модуль торговли на истории

In [ ]:
# количество акций
quantity = 100
# словарь значений MA
dict_ma = {"1h": 24, "10min": 144}

# по каждому таймфрейму
for timeframe in TIMEFRAMES:
    # создаем путь к файлу истории и статистики
    history_path = f"./history/{STOCK}/{timeframe}/{START_DATE}_{END_DATE}.csv"
    stat_path = f"./statistics/{STOCK}/{timeframe}/statistics_{STOCK}_{START_DATE}_{END_DATE}.csv"
    # читаем историю
    df = pd.read_csv(history_path, sep=",", encoding="utf-8")
    
    # создаем путь к директории статистики    
    if not os.path.exists(f"./statistics/{STOCK}/{timeframe}"):
        # создаем директорию
        os.makedirs(f"./statistics/{STOCK}/{timeframe}")
    # создаем датафрейм статистики
    df_stat = pd.DataFrame(columns=["TimeFrame", "Date", "Time", "Duration", "Position", "OpenPrice", "ClosePrice", "Quantity",
                             "Amount", "Lots"])

    # расчет средней
    df[f'ma{dict_ma[timeframe]}'] = df['close'].rolling(window=dict_ma[timeframe]).mean()
    # запускаем стратегию
    df_signals = strategy_takeover(df=df, ma=dict_ma[timeframe], tm=timeframe, add=True)
    # формируем дневник трейдера
    df_diary = diary_trading(signals=df_signals, df=df_stat, tm=timeframe, quantity=quantity)
    # сохраняем дневник трейдера
    df_diary.to_csv(stat_path, sep=",", encoding="utf-8", index=False)   
    display(df_diary)

### Модуль статистики торговли на истории

In [ ]:
# список дневников трейдинга
diaries = []
# получим список директорий
timeframes= [f for f in os.listdir(f"./statistics/{STOCK}")]
    
# список строк сводной статистики
rows = []
for timeframe in timeframes:
    if timeframe != "total":
        # заносим дневник в список дневников
        diary = pd.read_csv(f"./statistics/{STOCK}/{timeframe}/statistics_{STOCK}_{START_DATE}_{END_DATE}.csv", sep=",", encoding="utf-8")        
        # заносим компонент дневника в список дневников
        diaries.append(diary)
        # рассчитываем сводную статистику для таймфрейма
        stat_dict = statistics(diary=diary, stock=STOCK, tm=timeframe)
        # добавляем статистику в список
        rows.append(stat_dict)

# формируем файл сводной статистики
stat_total_path = f"./statistics/{STOCK}/total"
if not os.path.exists(stat_total_path):
    # создаем директорию
    os.makedirs(stat_total_path)
# формируем датафрейм сводной статистики
total_tbl = pd.DataFrame.from_dict(rows)
# если торговали по нескольким таймфреймам
if len(timeframes) > 1:
    # добавляем итоговую строку
    total_row = total_statistics(total_tbl=total_tbl, stock=STOCK)
    total_tbl = pd.concat([total_tbl, pd.DataFrame.from_dict([total_row])], ignore_index=True)
# сохраняем статистику в формат csv
total_tbl.to_csv(f"{stat_total_path}/total_stat_{STOCK}_{START_DATE}_{END_DATE}.csv", sep=",", encoding="utf-8", index=False)    


In [ ]:
# вывод статистики
display(total_tbl)